instantiate package

In [1]:
using Pkg
Pkg.instantiate()
using TeneT

┌ Info: OMEinsum loaded the CUDA module successfully
└ @ OMEinsum E:\juliapkg\packages\OMEinsum\vgB8s\src\cueinsum.jl:117


# 2D classical Ising model
The ''energy'' of a configuration $\sigma$ is given by the Hamiltonian function:

$H(\sigma) = -\sum_{\langle i~j\rangle} J \sigma_i \sigma_j$  ($J>0$)

The Boltzmann factor is:

$W_{i~j} = e^{-\beta E_{i~j}}$

We can solve this model by TeneT.jl in follow steps:

## Step 1: define M tensor
 <img src="../picture/build_M_tensor.png" width = "30%" height = "30%" align=center />

In [2]:
using TeneT: _arraytype
using OMEinsum
using Zygote

const isingβc = log(1+sqrt(2))/2

abstract type HamiltonianModel end

struct Ising <: HamiltonianModel 
    Ni::Int
    Nj::Int
    β::Float64
end

"""
    model_tensor(model::Ising, type)
return the  `MT <: HamiltonianModel` `type` tensor at inverse temperature `β` for  two-dimensional
square lattice tensor-network.
"""
function model_tensor(model::Ising, ::Val{:bulk})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    ham = Zygote.@ignore ComplexF64[-1. 1;1 -1]
    w = exp.(- β * ham)
    wsq = sqrt(w)
    m = ein"ia,ib,ic,id -> abcd"(wsq, wsq, wsq, wsq)

    M = Zygote.Buffer(m, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = m
    end
    return copy(M)
end

function model_tensor(model::Ising, ::Val{:mag})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    a = reshape(ComplexF64[1 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 -1], 2,2,2,2)
    cβ, sβ = sqrt(cosh(β)), sqrt(sinh(β))
    q = 1/sqrt(2) * [cβ+sβ cβ-sβ; cβ-sβ cβ+sβ]
    m = ein"abcd,ai,bj,ck,dl -> ijkl"(a,q,q,q,q)
    M = Zygote.Buffer(m, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = m
    end
    return copy(M)
end

function model_tensor(model::Ising, ::Val{:energy})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    ham = ComplexF64[-1 1;1 -1]
    w = exp.(-β .* ham)
    we = ham .* w
    wsq = sqrt(w)
    wsqi = wsq^(-1)
    e = (ein"ai,im,bm,cm,dm -> abcd"(wsqi,we,wsq,wsq,wsq) + ein"am,bi,im,cm,dm -> abcd"(wsq,wsqi,we,wsq,wsq) + 
        ein"am,bm,ci,im,dm -> abcd"(wsq,wsq,wsqi,we,wsq) + ein"am,bm,cm,di,im -> abcd"(wsq,wsq,wsq,wsqi,we)) / 2
    M = Zygote.Buffer(e, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = e
    end
    return copy(M)
end

model_tensor (generic function with 3 methods)

## Step 2: use TeneT.obs_env to get environment

In [3]:
using Random
Random.seed!(100)

β = 0.5
model = Ising(1, 1, β)
M = model_tensor(model, Val(:bulk))
env = TeneT.obs_env(M; χ=10, maxiter=10, miniter=1, 
                    updown=false, verbose=true, show_every=1);

↑ 

random initial 1×1 vumps_χ10 environment-> 

vumps@step: 1, error=Inf


vumps@step: 2, error=0.01730159744333545
vumps@step: 3, error=0.0006110217932778949
vumps@step: 4, error=8.525419213658111e-7
vumps@step: 5, error=1.486693561895049e-7
vumps@step: 6, error=3.5210780596961775e-8


vumps@step: 7, error=7.535505333349743e-9
vumps@step: 8, error=1.640536608963289e-9
vumps@step: 9, error=3.7020363904278236e-10
vumps done@step: 9, error=8.508814802635002e-11


## Step 3: use env to calculate observable

In [4]:
using TeneT: ALCtoAC
"""
    observable(env, model::MT, type)
return the `type` observable of the `model`. Requires that `type` tensor defined in model_tensor(model, Val(:type)).
"""
function observable(env, model::MT, ::Val{:Z}) where {MT <: HamiltonianModel}
    _, ALu, Cu, ARu, ALd, Cd, ARd, FL, FR, FLu, FRu = env
    atype = _arraytype(ALu)
    M   = atype(model_tensor(model, Val(:bulk)))
    χ,D,Ni,Nj = size(ALu)[[1,2,4,5]]
    
    z_tol = 1
    ACu = ALCtoAC(ALu, Cu)

    for j = 1:Nj,i = 1:Ni
        ir = i + 1 - Ni * (i==Ni)
        jr = j + 1 - Nj * (j==Nj)
        z = ein"(((adf,abc),dgeb),ceh),fgh ->"(FLu[:,:,:,i,j],ACu[:,:,:,i,j],M[:,:,:,:,i,j],FRu[:,:,:,i,j],conj(ACu[:,:,:,ir,j]))
        λ = ein"(acd,ab),(bce,de) ->"(FLu[:,:,:,i,jr],Cu[:,:,i,j],FRu[:,:,:,i,j],conj(Cu[:,:,ir,j]))
        z_tol *= Array(z)[]/Array(λ)[]
    end
    return z_tol^(1/Ni/Nj)
end

function observable(env, model::MT, type) where {MT <: HamiltonianModel}
    _, ALu, Cu, ARu, ALd, Cd, ARd, FL, FR, FLu, FRu = env
    χ,D,Ni,Nj = size(ALu)[[1,2,4,5]]
    atype = _arraytype(ALu)
    M     = atype(model_tensor(model, Val(:bulk)))
    M_obs = atype(model_tensor(model, type      ))
    obs_tol = 0
    ACu = ALCtoAC(ALu, Cu)
    ACd = ALCtoAC(ALd, Cd)

    for j = 1:Nj,i = 1:Ni
        ir = Ni + 1 - i
        obs = ein"(((adf,abc),dgeb),fgh),ceh -> "(FL[:,:,:,i,j],ACu[:,:,:,i,j],M_obs[:,:,:,:,i,j],ACd[:,:,:,ir,j],FR[:,:,:,i,j])
        λ = ein"(((adf,abc),dgeb),fgh),ceh -> "(FL[:,:,:,i,j],ACu[:,:,:,i,j],M[:,:,:,:,i,j],ACd[:,:,:,ir,j],FR[:,:,:,i,j])
        obs_tol += Array(obs)[]/Array(λ)[]
    end
    if type == Val(:mag)
        obs_tol = abs(obs_tol)
    end
    return obs_tol/Ni/Nj
end

"""
    magofβ(::Ising,β)
return the analytical result for the magnetisation at inverse temperature
`β` for the 2d classical ising model.
"""
magofβ(model::Ising) = model.β > isingβc ? (1-sinh(2*model.β)^-4)^(1/8) : 0.

magofβ

In [5]:
using Test

@testset "$(Ni)x$(Nj) ising forward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    β = 0.5
    model = Ising(Ni, Nj, β)
    M = atype(model_tensor(model, Val(:bulk)))
    env = obs_env(M; χ = 10, maxiter = 10, miniter = 1, 
         infolder = "./example/data/$model/", 
        outfolder = "./example/data/$model/", 
        updown = false, verbose = false, savefile = false
        )
    @test observable(env, model, Val(:Z)     ) ≈ 2.789305993957602
    @test observable(env, model, Val(:mag)   ) ≈ magofβ(model) 
    @test observable(env, model, Val(:energy)) ≈ -1.745564581767667
end

Test Summary:                | Pass  

Total
1x1 ising forward with Array |    3      3


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising forward with Array", Any[], 3, false, false)

## Step 4(optional): calculate energy by Zygote.gradient

In [6]:
@testset "$(Ni)x$(Nj) ising backward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    function logZ(β)
        model = Ising(1, 1, β)
        M = model_tensor(model, Val(:bulk))
        env = obs_env(M;χ = 10, maxiter = 10, miniter = 1, 
                        updown = false, verbose = false, savefile = false
                    )
        log(real(observable(env, model, Val(:Z))))
    end
    @test Zygote.gradient(β->-logZ(β), 0.5)[1] ≈ -1.745564581767667
end

Test Summary:                 | Pass  Total
1x1 ising backward with Array |    1      1


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising backward with Array", Any[], 1, false, false)

<font size=5>*try different Temperature $\beta$, bond dimension $\chi$, unit cell size $Ni \times Nj$ and CuArray with GPU!*</font>

# Finding the Ground State of infinite 2D Heisenberg model
The Heisenberg Hamiltonian function is:

$H = \sum_{\langle i~j\rangle} J^x S_i^x S_j^x + J^y S_i^y S_j^y + J^z S_i^z S_j^z $




In [7]:
"""
    Heisenberg(Jz::T,Jx::T,Jy::T) where {T<:Real}
    
return a struct representing the heisenberg model with magnetisation fields `Jz`, `Jx` and `Jy`..
"""
struct Heisenberg{T<:Real} <: HamiltonianModel
    Jx::T
    Jy::T
    Jz::T
end
Heisenberg() = Heisenberg(1.0,1.0,1.0)


Sx = Float64[0 1; 1 0]/2
Sy = ComplexF64[0 -1im; 1im 0]/2
Sz = Float64[1 0; 0 -1]/2
"""
    hamiltonian(model::Heisenberg)

return the heisenberg hamiltonian for the `model` as a two-site operator.
"""
function hamiltonian(model::Heisenberg)
    h = model.Jx * ein"ij,kl -> ijkl"(Sx, Sx) +
        model.Jy * ein"ij,kl -> ijkl"(Sy, Sy) +
        model.Jz * ein"ij,kl -> ijkl"(Sz, Sz)
end

hamiltonian

The algorithm variationally minimizes the energy of a Heisenberg model on a two-dimensional infinite lattice using a form of gradient descent.

## Step 1: initial iPEPS tensor

In [8]:
using LinearAlgebra: norm

"""
    init_ipeps(model::HamiltonianModel; D::Int, χ::Int, tol::Real, maxiter::Int)
Initial `bcipeps` and give `key` for use of later optimization. The key include `model`, `D`, `χ`, `tol` and `maxiter`. 
The iPEPS is random initial if there isn't any calculation before, otherwise will be load from file `/data/model_D_chi_tol_maxiter.jld2`
"""
function init_ipeps(model::HamiltonianModel; folder::String="./data/", atype = Array, Ni::Int, Nj::Int, D::Int, χ::Int, tol::Real=1e-10, maxiter::Int=10, miniter::Int=1, verbose = true)
    folder *= "$(Ni)x$(Nj)/$(model)/"
    mkpath(folder)
    chkp_file = folder*"D$(D)_chi$(χ)_tol$(tol)_maxiter$(maxiter)_miniter$(miniter).jld2"
    if isfile(chkp_file)
        A = load(chkp_file)["bcipeps"]
        verbose && println("load BCiPEPS from $chkp_file")
    else
        A = rand(ComplexF64,D,D,D,D,2,Ni,Nj)
        verbose && println("random initial BCiPEPS $chkp_file")
    end
    A /= norm(A)
    key = (folder, model, atype, Ni, Nj, D, χ, tol, maxiter, miniter)
    return A, key
end

init_ipeps

In [9]:
model = Heisenberg(-1.0, -1.0, 1.0)
A, key = init_ipeps(model; Ni=1, Nj=1, D=2, χ=10);

random initial BCiPEPS ./data/1x1/Heisenberg{Float64}(-1.0, -1.0, 1.0)/D2_chi10_tol1.0e-10_maxiter10_miniter1.jld2

## Step 2: contract the tensor network to get the energy

In [10]:
using OMEinsumContractionOrders

"""
    oc_H, oc_V = optcont(D::Int, χ::Int)
optimise the follow two einsum contractions for the given `D` and `χ` which are used to calculate the energy of the 2-site hamiltonian:
```
                                            a ────┬──── c          
a ────┬──c ──┬──── f                        │     b     │  
│     b      e     │                        ├─ e ─┼─ f ─┤  
├─ g ─┼─  h ─┼─ i ─┤                        g     h     i 
│     k      n     │                        ├─ j ─┼─ k ─┤ 
j ────┴──l ──┴──── o                        │     m     │ 
                                            l ────┴──── n 
```
where the central two block are six order tensor have extra bond `pq` and `rs`
"""
function optcont(D::Int, χ::Int)
    sd = Dict('a' => χ, 'b' => D^2,'c' => χ, 'e' => D^2, 'f' => χ, 'g' => D^2, 'h' => D^2, 'i' => D^2, 'j' => χ, 'k' => D^2, 'l' => χ, 'n' => D^2, 'o' => χ, 'p' => 2, 'q' => 2, 'r' => 2, 's' => 2)
    # for seed =20:100
    seed = 60
	Random.seed!(seed)
	# oc_H = optimize_code(ein"agj,abc,gkhbpq,jkl,fio,cef,hniers,lno -> pqrs", sd, TreeSA())
    oc_H = ein"(((agj,abc),gkhbpq),jkl),(((fio,cef),hniers),lno) -> pqrs"
	print("Horizontal Contraction Complexity(seed=$(seed))",OMEinsum.timespace_complexity(oc_H,sd),"\n")
    
    sd = Dict('a' => χ, 'b' => D^2, 'c' => χ, 'e' => D^2, 'f' => D^2, 'g' => χ, 'h' => D^2, 'i' => χ, 'j' => D^2, 'k' => D^2, 'l' => χ, 'm' => D^2, 'n' => χ, 'r' => 2, 's' => 2, 'p' => 2, 'q' => 2)
    # oc_V = optimize_code(ein"abc,aeg,ehfbpq,cfi,gjl,jmkhrs,ikn,lmn -> pqrs", sd, TreeSA())
    oc_V = ein"(((abc,aeg),ehfbpq),cfi),(gjl,(jmkhrs,(ikn,lmn))) -> pqrs"
    print("Vertical Contraction Complexity(seed=$(seed))",OMEinsum.timespace_complexity(oc_V,sd),"\n") 
    oc_H, oc_V
end

"""
    energy(h, bcipeps; χ, tol, maxiter)
return the energy of the `bcipeps` 2-site hamiltonian `h` and calculated via a
BCVUMPS with parameters `χ`, `tol` and `maxiter`.
"""
function energy(h, A, oc, key; verbose = true, savefile = true)
    folder, model, atype, Ni, Nj, D, χ, tol, maxiter, miniter = key
    ap = ein"abcdeij,fghmnij->afbgchdmenij"(A, conj(A))
    ap = reshape(ap, D^2, D^2, D^2, D^2, 2, 2, Ni, Nj)
    M = ein"abcdeeij->abcdij"(ap)

    env = obs_env(M; χ = χ, tol = tol, maxiter = maxiter, miniter = miniter, verbose = verbose, savefile = savefile, infolder = folder, outfolder = folder)
    e = expectationvalue(h, ap, env, oc, key)
    return e
end

function expectationvalue(h, ap, env, oc, key)
    _, ALu, Cu, ARu, ALd, Cd, ARd, FL, FR, FLu, FRu = env
    folder, model, atype, Ni, Nj, D, χ, tol, maxiter, miniter = key
    oc_H, oc_V = oc
    ACu = ALCtoAC(ALu, Cu)
    ACd = ALCtoAC(ALd, Cd)

    etol = 0
    for j = 1:Nj, i = 1:Ni
        println("===========$i,$j===========")
        ir = Ni + 1 - i
        jr = j + 1 - (j==Nj) * Nj
        lr = oc_H(FL[:,:,:,i,j],ACu[:,:,:,i,j],ap[:,:,:,:,:,:,i,j],ACd[:,:,:,ir,j],FR[:,:,:,i,jr],ARu[:,:,:,i,jr],ap[:,:,:,:,:,:,i,jr],ARd[:,:,:,ir,jr])
        e = Array(ein"pqrs, pqrs -> "(lr,h))[]
        n =  Array(ein"pprr -> "(lr))[]
        println("Horizontal energy = $(e/n)")
        etol += e/n

        ir  =  i + 1 - (i==Ni) * Ni
        irr = Ni - i + (i==Ni) * Ni
        lr = oc_V(ACu[:,:,:,i,j],FLu[:,:,:,i,j],ap[:,:,:,:,:,:,i,j],FRu[:,:,:,i,j],FL[:,:,:,ir,j],ap[:,:,:,:,:,:,ir,j],FR[:,:,:,ir,j],ACd[:,:,:,irr,j])
        e = Array(ein"pqrs, pqrs -> "(lr,h))[]
        n = Array(ein"pprr -> "(lr))[]
        println("Vertical energy = $(e/n)")
        etol += e/n
    end

    println("e = $(etol/Ni/Nj)")
    return etol/Ni/Nj
end

expectationvalue (generic function with 1 method)

In [11]:
oc = optcont(2, 10)
h = hamiltonian(model)
energy(h, A, oc, key; verbose = true, savefile = true);

Horizontal Contraction Complexity(seed=20)(18.501837184902293, 12.643856189774723)

## Step 3: optimise the ipeps by gradient

In [ ]:
using FileIO
using LineSearches
using Optim

"""
    optimiseipeps(A::AbstractArray, key; f_tol = 1e-6, opiter = 100, verbose= false, optimmethod = LBFGS(m = 20))

return the tensor `A'` that describes an ipeps that minimises the energy of the
two-site hamiltonian `h`. The minimization is done using `Optim` with default-method
`LBFGS`. Alternative methods can be specified by loading `LineSearches` and
providing `optimmethod`. Other options to optim can be passed with `optimargs`.
The energy is calculated using vumps with key include parameters `χ`, `tol` and `maxiter`.
"""
function optimiseipeps(A::AbstractArray, key; f_tol = 1e-6, opiter = 100, verbose= false, optimmethod = LBFGS(m = 20))
    folder, model, atype, Ni, Nj, D, χ, tol, maxiter, miniter = key

    h = hamiltonian(model)
    oc = optcont(D, χ)
    f(x) = real(energy(h, x, oc, key))
    g(x) = Zygote.gradient(f,x)[1]
    res = optimize(f, g, 
        A, optimmethod,inplace = false,
        Optim.Options(f_tol=f_tol, iterations=opiter,
        extended_trace=true,
        callback=os->writelog(os, key)),
        )
    return res
end

"""
    writelog(os::OptimizationState, key=nothing)

return the optimise infomation of each step, including `time` `iteration` `energy` and `g_norm`, saved in `/data/model_D_chi_tol_maxiter.log`. Save the final `ipeps` in file `/data/model_D_chi_tol_maxiter.jid2`
"""
function writelog(os::OptimizationState, key=nothing)
    message = "$(round(os.metadata["time"],digits=2))   $(os.iteration)   $(os.value)   $(os.g_norm)\n"

    printstyled(message; bold=true, color=:red)
    flush(stdout)

    folder, model, atype, Ni, Nj, D, χ, tol, maxiter, miniter = key
    !(isdir(folder)) && mkdir(folder)
    if !(key === nothing)
        logfile = open(folder*"D$(D)_χ$(χ)_tol$(tol)_maxiter$(maxiter).log", "a")
        write(logfile, message)
        close(logfile)
        save(folder*"D$(D)_χ$(χ)_tol$(tol)_maxiter$(maxiter).jld2", "bcipeps", os.metadata["x"])
    end
    return false
end

In [ ]:
optimiseipeps(A, key; f_tol = 1e-6, opiter = 100, verbose= false, optimmethod = LBFGS(m = 20))

In [ ]:
1+1